In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# import Config
from user_agent import generate_user_agent, generate_navigator
from selenium.common.exceptions import NoSuchElementException

options = webdriver.ChromeOptions()
# user_agent= "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
# user_agent = generate_user_agent(os=('mac'), device_type='desktop')
user_agent = generate_user_agent(device_type='desktop')
options.add_argument('--incognito')
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-setid-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('user-agent=' + user_agent)
# chrome_path = "/Users/baaam771/Downloads/chromedriver 2"
# driver = webdriver.Chrome(chrome_path)
# driver = webdriver.Chrome(chrome_path, options=options)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# driver = webdriver.Chrome(executable_path=Config.CONFIG['CHROMEPATH'],options=options)
import re
import pandas as pd
from selenium.webdriver.common.by import By


import time

titles = []
singers = []
numbers = []
LYRIC = []
LYRIC_clean = []


for i in range(1,55152,50):
    str_i = str(i)

    driver.get("https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex="+str_i)

    time.sleep(0.1)
    title=driver.find_elements(By.CLASS_NAME, 'ellipsis.rank01')

    for a in title:
        titles.append(a.text)

    # print(titles)

    singer =driver.find_elements(By.CLASS_NAME, 'ellipsis.rank02')

    for b in singer:
        singers.append(b.text)

    # print(singers)

    number=driver.find_elements(By.CSS_SELECTOR, 'div.ellipsis.rank01 > span > a')

    for c in number:
        numbers.append(c.get_attribute('href')[-10:-2])
    # print(numbers)
    # time.sleep(2)

for i in numbers:
    driver.get("https://www.melon.com/song/detail.htm?songId=" + i)
    try:
        lyric = driver.find_element(By.CLASS_NAME, 'lyric')
        LYRIC.append(lyric.text)
    except NoSuchElementException:
        LYRIC.append("blank")
    # lyric = driver.find_element(By.CLASS_NAME, 'lyric')
    # LYRIC.append(lyric.text)

for i in LYRIC:
    LYRIC_clean.append(i.replace("\n", " "))



# print(titles)
# print(singers)
# print(numbers)
# print('수집완료')

# for i in numbers:
#     driver.get("https://www.melon.com/song/detail.htm?songId=" + i)
#     lyric = driver.find_element(By.CLASS_NAME, 'lyric')
#     LYRIC.append(lyric.text)

# \n 제거
# for i in LYRIC:
#     LYRIC_clean.append(i.replace("\n", " "))

df = pd.DataFrame({"제목":titles, "가수":singers,"가사": LYRIC_clean})

df.to_csv("music_data_balad_new_2.csv", mode='w')
df.to_csv("music_data_balad_new_1.csv", mode='a', header=False)

df.to_excel("music_data_balad_new_2.xlsx", encoding='utf-8')